# ✨ Welcome to the OSCAR from a Jupyter Notebook Usage Tutorial 🚀

In this documentation, we will step-by-step set up our environment and provide simple example use cases to help you understand the basics of working with OSCAR clusters from a Jupyter Notebook. 🖥️📡   

This tutorial will focus on:
- How to set up the environment of the Jupyter Notebook
- How to define a service
- How to create a service in OSCAR
- How to call synchronous services
- How to call asynchronous services
- How to upload and download files from the service's bucket
- How to delete services

Let's get started! 🚀💡  


## First, set up the environment

Open the terminal and execute `sudo su` (if you are not already a root user) and `./setupscript.sh`. This script will configure the OpenID client and initialize an authentication process in which you will obtain a device code authentication that you need to insert on the EGI authentication portal. 

Follow the instructions:
- Introduce the code in this webside https://aai.egi.eu/device.
- Return to the terminal and write your password twice.

The short name for the authenticated user by default is "oscar-egi".

## 🛠️ Service Definition  

To create a service on the cluster, we need to define it's configuration using a **YAML** file in **FDL** format.  

We also need:  
📌 A **Docker image** that includes the AI model or other functionality and contains all required libraries and data to perform its execution. 🐳  
📌 A **shell-script** to be executed inside the container created out of the Docker image for each service invocation. 📜  

📖 Learn more about **FDL** in the [OSCAR documentation](https://docs.oscar.grycap.net/fdl/).  
🎨 For an easier graphical configuration, check out the **[FDL Composer](https://docs.oscar.grycap.net/fdl-composer/)**.    


Next, we will provide guidance for two basic use cases:  

✔️ **Synchronous Service (Cowsay 🐄💬)** ⏳  
   - Receives a text input 📝  
   - Returns text output 📤  

✔️ **Asynchronous Service (Plant Classification🌿)** ⏳🔄  
   - Triggers when a file is saved on the input storage 📂  
   - Processes it and sends the output to the same or another storage 📬 

**_NOTE:_** You can find more service examples in the [examples](https://github.com/grycap/oscar/tree/v3.3.3/examples) section on GitHub. 

### 🐮 Cowsay  

A **synchronous** service that receives an input in **JSON** and returns a text response:

📥 **INPUT:** 
```json
    {"message": "Hi there"}
```
📤 **OUTPUT:** 
```
     __________
    < Hi there >
     ----------
            \   ^__^
             \  (oo)\_______
                (__)\       )\/\
                    ||----w |
                    ||     ||

```

#### 📜 FDL Definition  

To define a **basic FDL configuration file**, we need to specify:  

📌 **Service Name** → The unique name of the service. 🏷️  
📌 **CPU Cores (`cpu`)** → The number of cores required for execution. ⚡  
📌 **Memory (`memory`)** → The amount of RAM needed (in **Mi** or **Gi**). 💾  
📌 **Docker Image (`image`)** → The container image essential for the service. 🐳  
📌 **Execution Script (`script`)** → The script that will run inside the instance. 📜  

🔹 Aditionaly we set the log level to **critical** to avoid supervisor logs in response.

### 🌱 Plant Classification  

An **asynchronous** service that automatically detects and classifies plants from images. It receives an image and returns it after processing.

📥 **INPUT** → Stored in **MinIO**:

![image](img/plant-input.jpg)  

📤 **OUTPUT** → Saved in **MinIO** or another storage:

![image](img/plant-output.jpg)   

#### 📜 FDL Definition  

Since this service is **asynchronous**, we need to specify:

📌 **Input Storage (`input`)** 📂 → Triggers the execution  
📌 **Output Storage (`output`)** 📂 → Stores the processed results  

🔹 We **do not** set the log level to **critical** because we won’t receive the response directly.


## ⚙️ Service Creation  

After setting up the environment and defining our services, we can now **deploy** them on our **OSCAR cluster**. 🚀  

### 📚 Import Libraries  

To interact with **OSCAR** from a **Jupyter Notebook**, we will use the **OSCAR Python client library**. 

📖 Learn more about it on:  
🔹 [GitHub](https://github.com/grycap/oscar_python) 🛠️  
🔹 [PyPi](https://pypi.org/project/oscar-python/) 📦  

📌 Check the **OSCAR [OpenAPI documentation](https://docs.oscar.grycap.net/api/)** to understand the structure of the returned data. 

**_NOTE:_** This tutorial uses **version 1.2.1** of the Python library. 

In [1]:
from oscar_python.client import Client
import json
import os

### 🔑 Create the Client with Credentials  

To interact with the **OSCAR cluster**, we must create a client instance and provide:  

🔹 **Cluster Identification Name (`cluster_id`)** → Used in the service configuration file to identify the cluster.  
🔹 **OSCAR Cluster Endpoint (`endpoint`)** → The environment variable is set to the current cluster where the notebook is created.  
🔹 **User Identification Tokens (`shortname`)** → Configured during environment setup with the default short name **"oscar-egi"**. 


In [2]:
options_oidc_auth = {'cluster_id': 'cluster-id',
                     'endpoint': os.environ["OSCAR_ENDPOINT"],
                     'shortname': 'oscar-egi',
                     'ssl': 'True'}

client = Client(options=options_oidc_auth)

### 🚀 Create Services  

To deploy services in the cluster, simply call the `create_service()` function and provide **service configuration file**:  

```python
    client.create_service("service_config.yaml")
```

In [3]:
try:
    client.create_service("cowsay-sync.yaml")
    client.create_service("plant-async.yaml")
    print("Services created")
except Exception as err:
    print("Failed with: ", err)

Services created


#### 📋 Print the List of Available Services  

To check which services are currently deployed in the cluster, use the following command:  

```python
    client.list_services()
```
🔹 This function returns an HTTP response containing a JSON list of available services.

In [4]:
try:
    services = client.list_services()
    print("Services:")
    if services.status_code == 200:
        # Print the name of the services on the cluster
        for value in services.json():
            print("> " + value["name"])
except Exception as err:
    print("Failed with: ", err)

Services:
> cowsay
> cowsay-sync-test
> juno7e3b98
> plant-classification-async-test
> plants


---  

🔹 You can also view the entire **JSON response** for more details (to improve readability, we format the output as a structured string).

In [ ]:
try:
    services = client.list_services()
    if services.status_code == 200:
        json_object = json.loads(services.text)
        json_formatted_str = json.dumps(json_object, indent=2)
        print(json_formatted_str)
except Exception as err:
    print("Failed with: ", err)